## Подготовка данных

Импортируем библиотеки

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, r2_score, confusion_matrix, recall_score
from sklearn.metrics import roc_curve, precision_score
from sklearn.utils import shuffle

pd.options.mode.chained_assignment = None

Загрузим данные, и посмотрим содержимое

In [2]:
data = pd.read_csv('/datasets/Churn.csv')
display(data.head(10))

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


Посмотрим инфо

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


Есть пропуски в столбце Tenure- сколько лет клиент пользуется услугами банка. Посмотрим, как выглядят строки с пропуском в этом столбце

In [4]:
display(data[data['Tenure'].isna()])

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
30,31,15589475,Azikiwe,591,Spain,Female,39,NaN,0.00,3,1,0,140469.38,1
48,49,15766205,Yin,550,Germany,Male,38,NaN,103391.38,1,0,1,90878.13,0
51,52,15768193,Trevisani,585,Germany,Male,36,NaN,146050.97,2,0,0,86424.57,0
53,54,15702298,Parkhill,655,Germany,Male,41,NaN,125561.97,1,0,0,164040.94,1
60,61,15651280,Hunter,742,Germany,Male,35,NaN,136857.00,1,0,0,84509.57,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9944,9945,15703923,Cameron,744,Germany,Male,41,NaN,190409.34,2,1,1,138361.48,0
9956,9957,15707861,Nucci,520,France,Female,46,NaN,85216.61,1,1,0,117369.52,1
9964,9965,15642785,Douglas,479,France,Male,34,NaN,117593.48,2,0,0,113308.29,0
9985,9986,15586914,Nepean,659,France,Male,36,NaN,123841.49,2,1,0,96833.00,0


In [5]:
display(data['Tenure'].unique())

array([ 2.,  1.,  8.,  7.,  4.,  6.,  3., 10.,  5.,  9.,  0., nan])

Между клиентами с пропусками в этом столбце нет ничего общего, у остальных диапазон значений- от 0 до 10 лет.

Удалять строки с пропусками нежелательно- это примерно 10% от датафрейма, и остальные данные по этим клиентам в порядке. Заменим пропуски медианным значением.

In [6]:
data['Tenure'] = data['Tenure'].fillna(data['Tenure'].median())

Проверим результат

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


Пропусков больше нет. Для анализа, нам не нужны столбцы с индексом строки, id клиента  и его фамилией- эти данные, очевидно, не влияют на уход клиента из банка. Удалим их.

In [8]:
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
display(data.head(5))

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


Проверим дубликаты

In [9]:
display(data.duplicated().sum())

0

Дубликатов нет.

Подготовим данные методом OHE.

In [10]:
data = pd.get_dummies(data, drop_first=True)
display(data.head(5))

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0


Данные подготовлены, можно перейти к разбвивке на выборки: обучающую, валидационную и тестовую, по схеме 3:1:1.

Разделим датафрейм по признакам, целевой для нас- Exited.

In [11]:
features = data.drop('Exited', axis=1)
target = data['Exited']
display(features.head(5))
display(target.head(5))

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,1,0


0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

Успешно. Теперь выделим тестовую выборку, на нее приходится 20% от всех данных

In [12]:
data_features, features_test, data_target, target_test = train_test_split(features, 
                                                                          target, test_size=0.2, random_state=12345)

data_features и data_target- это сумма будущих обучающей и валидацинной выборки. Чтобы выделить из нее валидационную, согласно заявленной нами схеме 3:1:1, нам нужно выделить 25%

In [13]:
features_train, features_valid, target_train, target_valid = train_test_split(data_features, data_target, 
                                                                              test_size=0.25, random_state=12345)

Проверим результат

In [14]:
display(features_test.shape[0], features_valid.shape[0], features_train.shape[0])

2000

2000

6000

Успешно.

Осталось масштабировать количественные признаки. Вот их перечень:

CreditScore — кредитный рейтинг

Age — возраст

Tenure — сколько лет человек является клиентом банка

Balance — баланс на счёте

NumOfProducts — количество продуктов банка, используемых клиентом

EstimatedSalary — предполагаемая зарплата

Для удобства, создадим список столбцов с этими признаками. Далее, проведем обучение на обучающей выборке, и масштабируем в ней количественные признаки

In [15]:
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']

scaler = StandardScaler()
scaler.fit(features_train[numeric])

features_train[numeric] = scaler.transform(features_train[numeric])
display(features_train.head(5))

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
492,-0.134048,-0.078068,-0.369113,0.076163,0.816929,0,1,0.331571,0,0,0
6655,-1.010798,0.494555,-0.007415,0.136391,-0.896909,1,1,-0.727858,0,0,1
4287,0.639554,1.353490,-1.454209,0.358435,-0.896909,1,1,-0.477006,1,0,1
42,-0.990168,2.116987,-1.092511,0.651725,-0.896909,1,1,-0.100232,0,0,0
8178,0.567351,0.685430,0.715982,0.813110,0.816929,1,1,0.801922,0,0,0


Теперь масштабируем данные в валидационной и тестовой выборке

In [16]:
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])
display(features_valid.head(5))
display(features_test.head(5))

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
2358,0.175393,0.399118,-1.454209,1.385698,-0.896909,0,1,-1.466761,0,0,1
8463,-1.299609,0.971741,-1.092511,-1.232442,-0.896909,1,0,0.254415,0,1,1
163,0.711757,-0.268942,-1.092511,-1.232442,0.816929,1,1,0.122863,0,1,0
3074,-0.391916,0.494555,0.354284,0.672529,-0.896909,1,0,0.585847,1,0,0
5989,0.165078,1.353490,1.801078,0.536522,-0.896909,0,0,1.462457,0,0,0


,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
7867,-0.123733,0.685430,-0.730812,-1.232442,-0.896909,1,1,0.980212,0,1,0
1402,1.083087,-0.937002,1.077681,0.858518,-0.896909,1,0,-0.390486,0,0,1
8606,1.598822,0.303681,-0.007415,-1.232442,0.816929,1,1,-0.435169,0,1,1
8885,0.165078,0.589993,-0.369113,0.412100,0.816929,1,1,1.017079,0,1,1
6494,0.484834,-1.032439,0.715982,-1.232442,0.816929,1,1,-1.343558,0,0,1


Выводы:

1) Импортировали библиотеки, загрузили данные
2) Заполнили пропуски медианным значением
3) Убрали ненужные для задачи столбцы
4) Обработали датафрейм методом OHE
5) Выполнили разбивку данных на обучающую, валидационную, и тестовую выборку по сотношению 3:1:1

## Исследование задачи

Наш целевой признак- Exited. Это категориальный признак. Следовательно, нам подойдут методы "решающее дерево", "случайный лес", "логистическая регрессия". Проверим их точность.

In [17]:
model_decision_tree = DecisionTreeClassifier(random_state=123)
decision_tree_score = model_decision_tree.fit(features_train, target_train).score(features_valid, target_valid)

model_random_forest = RandomForestClassifier(random_state=123)
random_forest_score = model_random_forest.fit(features_train, target_train).score(features_valid, target_valid)

model_logistic_regression = LogisticRegression(random_state=123)
logistic_regression_score = model_logistic_regression.fit(features_train, target_train).score(features_valid, target_valid)
display('Точность решающего дерева:', decision_tree_score)
display('Точность случайного леса:', random_forest_score)
display('Точность логистической регрессии:', logistic_regression_score)

'Точность решающего дерева:'

0.7945

'Точность случайного леса:'

0.8615

'Точность логистической регрессии:'

0.8145

На стандартных гиперпараметрах, лучшую точность показывает случайный лес.

Проверим баланс классов.

In [18]:
display(target_train.value_counts(normalize = 1))
display(target_valid.value_counts(normalize = 1))

0    0.796833
1    0.203167
Name: Exited, dtype: float64

0    0.8045
1    0.1955
Name: Exited, dtype: float64

Примерно 80% ответов- 0. Классы несбалансированы, это может сказаться на результате.

Посмотрим для каждой модели значение f1-меры, AUC-ROC и матрицу ошибок.

In [19]:
decision_tree_prediction = model_decision_tree.predict(features_valid)
display(confusion_matrix(target_valid, decision_tree_prediction))
display(f1_score(target_valid, decision_tree_prediction))
display(roc_auc_score(target_valid, decision_tree_prediction))

array([[1391,  218],
       [ 193,  198]])

0.49070631970260226

0.6854529906106794

Значение f1-меры, у решающего дерева, ниже целевого. Количество ложно-позитивных предсказаний выше, чем количество истинно- позитивных. Модель склонна выдавать позитивные предсказания.

AUC-ROC выше 0.5

In [20]:
random_forest_prediction = model_random_forest.predict(features_valid)
display(confusion_matrix(target_valid, random_forest_prediction))
display(f1_score(target_valid, random_forest_prediction))
display(roc_auc_score(target_valid, random_forest_prediction))

array([[1548,   61],
       [ 216,  175]])

0.5582137161084529

0.7048292930272333

Значение f1-меры у случайного леса также ниже целевого. При этом, качество предсказаний выше, чем у решающего дерева, что видно и по матрице ошибок. Однако, у модели есть уклон в сторону ложно-негативных предсказаний.

Значение AUC-ROC также выросло по сравнению с моделью решающего дерева.

In [21]:
logistic_regression_prediction = model_logistic_regression.predict(features_valid)
display(confusion_matrix(target_valid, logistic_regression_prediction))
display(f1_score(target_valid, logistic_regression_prediction))
display(roc_auc_score(target_valid, logistic_regression_prediction))

array([[1549,   60],
       [ 311,   80]])

0.30131826741996237

0.5836566690880421

Модель имеет очень большой перекос в сторону ложно-негативных предсказаний.

Значение AUC-ROC существенно ниже, чем у прошлых моделей, но все равно выше 0.5

Сравним результаты моделей с константной моделью (она будет во всех случаях давать ответ: 0)

In [22]:
target_predict_constant = pd.Series([0]*len(target_valid))
accuracy_score_constant = accuracy_score(target_valid, target_predict_constant)
display(accuracy_score_constant)

0.8045

Точность выше, чем у решающего дерева, и лишь немногим ниже, чем у логистической регрессии. Только случайный лес показывает заметно лучший результат по сравнению с константной моделью.

При этом, логистическая регрессия склонна выдавать негативный ответ, что объясняется несбалансированностью классов. Перейдем к борьбе с дисбалансом.

Выводы:

1) Определились с выбором моделей

2) Проверили метрики

3) Проверили баланс классов

4) Обнаружили, что во всех трех моделях, чем выше значение F1-меры, тем выше значение AUC-ROC

4) Сравнили результаты метрик с константной моделью.

## Борьба с дисбалансом

Примерное соотношение негативных и позитивных ответов- 80:20. Следовательно, нам нужно увеличить количество позитивных ответов в выборке в 4 раза.

Разделим выборку на негативные и позитивные объекты, увеличим выборку с позитивными объектами в 4 раза, объединим выборки и смешаем объекты в получившейся выборке.

In [23]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_train_upsampled, target_train_upsampled = upsample(features_train, target_train, 4)
display(target_train_upsampled.value_counts(normalize = 1))

1    0.504919
0    0.495081
Name: Exited, dtype: float64

Провели апсэмлинг, теперь посмотрим, как это скажется на работе моделей.

In [24]:
model_decision_tree = DecisionTreeClassifier(random_state=123)
decision_tree_score = model_decision_tree.fit(features_train_upsampled, target_train_upsampled).score(
    features_valid, target_valid)

model_random_forest = RandomForestClassifier(random_state=123)
random_forest_score = model_random_forest.fit(features_train_upsampled, target_train_upsampled).score(
    features_valid, target_valid)

model_logistic_regression = LogisticRegression(random_state=123)
logistic_regression_score = model_logistic_regression.fit(features_train_upsampled, target_train_upsampled).score(
    features_valid, target_valid)
display('Точность решающего дерева:', decision_tree_score)
display('Точность случайного леса:', random_forest_score)
display('Точность логистической регрессии:', logistic_regression_score)

'Точность решающего дерева:'

0.798

'Точность случайного леса:'

0.851

'Точность логистической регрессии:'

0.703

У логистической регрессии сильно упала точность. Посмотрим на прочие метрики у всех трех моделей.

In [25]:
decision_tree_prediction = model_decision_tree.predict(features_valid)
display(confusion_matrix(target_valid, decision_tree_prediction))
display(f1_score(target_valid, decision_tree_prediction))
display(roc_auc_score(target_valid, decision_tree_prediction))

array([[1415,  194],
       [ 210,  181]])

0.4725848563968668

0.6711719086532119

In [26]:
random_forest_prediction = model_random_forest.predict(features_valid)
display(confusion_matrix(target_valid, random_forest_prediction))
display(f1_score(target_valid, random_forest_prediction))
display(roc_auc_score(target_valid, random_forest_prediction))

array([[1498,  111],
       [ 187,  204]])

0.5779036827195467

0.726376091009809

In [27]:
logistic_regression_prediction = model_logistic_regression.predict(features_valid)
display(confusion_matrix(target_valid, logistic_regression_prediction))
display(f1_score(target_valid, logistic_regression_prediction))
display(roc_auc_score(target_valid, logistic_regression_prediction))

array([[1135,  474],
       [ 120,  271]])

0.4771126760563381

0.6992508571510319

У логистичкесой регрессии показатель f1-меры сильно вырос. На прочих моделях, балансирование классов сказалось не так сильно.

Теперь попробуем, напротив, уменьшить в 4 раза количество негативных ответов. Аналогично, создадим функцию:

In [28]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_train_downsampled, target_train_downsampled = downsample(features_train, target_train, 0.25)
display(target_train_downsampled.value_counts(normalize = 1))

1    0.504971
0    0.495029
Name: Exited, dtype: float64

Проверим, что будет получаться на выборках после даунсэмлинга. Заново обучим модели на новой выборке

In [29]:
model_decision_tree = DecisionTreeClassifier(random_state=123)
decision_tree_score = model_decision_tree.fit(features_train_downsampled, target_train_downsampled).score(
    features_valid, target_valid)

model_random_forest = RandomForestClassifier(random_state=123)
random_forest_score = model_random_forest.fit(features_train_downsampled, target_train_downsampled).score(
    features_valid, target_valid)

model_logistic_regression = LogisticRegression(random_state=123)
logistic_regression_score = model_logistic_regression.fit(features_train_downsampled, target_train_downsampled).score(
    features_valid, target_valid)
display('Точность решающего дерева:', decision_tree_score)
display('Точность случайного леса:', random_forest_score)
display('Точность логистической регрессии:', logistic_regression_score)

'Точность решающего дерева:'

0.694

'Точность случайного леса:'

0.7685

'Точность логистической регрессии:'

0.7005

В целом, результаты ниже, чем у апсэмлинга. Проверим метрики даунсэмлинга у модели случайного леса.

In [30]:
random_forest_prediction = model_random_forest.predict(features_valid)
display(confusion_matrix(target_valid, random_forest_prediction))
display(f1_score(target_valid, random_forest_prediction))
display(roc_auc_score(target_valid, random_forest_prediction))

array([[1250,  359],
       [ 104,  287]])

0.553519768563163

0.7554476974944326

Результат ниже, чем при апсэмлинге.
Самый высокий показатель f1-меры у модели случайного леса: 0.57, после обучения на выборке с апсэмплингом.

Это очень близко к целевому значению не ниже 0.59. 
    
Попробуем улучшить результат этой модели подбором гиперпараметров.

Выводы:

1) Масштабировали выборку, методом апсэмлинга

2) Масштабировали выборку методом даунсэмплинга. Обнаружили, что результат обучения модели на выборке после апсэмплинга лучше.

3) Проверили метрики на моделях, обученных по новой выборке. Определились с выбором итоговой модели.

## Тестирование модели

Создадим цикл, в котором будут перебираться гиперпараметры, и попробуем найти наилучший результат.

In [31]:
best_params = []
best_f1_score = 0
#создаем циклы, которые будут перебирать значения параметров.
#цикл максимальной глубины дерева
for max_depth in range(2,15):
#цикл минимального колчичества объектов в узле    
    for min_samples_split in range(2,8):
#цикл минимального количества объектов в листе        
        for min_samples_leaf in range(1,15):
#цикл количества деревьев 
            for n_estimators in (1,20):
                # Проверим модель с текущими параметрами
                model_random_forest = RandomForestClassifier(random_state = 123,
                                                      max_depth = max_depth,
                                                      min_samples_split = min_samples_split,
                                                      min_samples_leaf = min_samples_leaf,
                                                      n_estimators = n_estimators)                                                    
                # обучение модели на тренировочной выборке
                model_random_forest.fit(features_train_upsampled, target_train_upsampled)
                # поиск предсказаний модели на валидационной выбоке
                predictions = model_random_forest.predict(features_valid)
                # вычисление точности модели методом accuracy_score
                model_f1_score = f1_score(target_valid, predictions)
                # если текущее значение точности выше предыдущего лучшего значения, 
                # сохраняем параметры модели и текущую точность
                if model_f1_score > best_f1_score:
                    best_params = [max_depth, min_samples_split, min_samples_leaf, n_estimators]
                    best_f1_score = model_f1_score

display('Самый высокий показатель f1-меры:', best_f1_score)
display('Лучшие значения параметров max_depth, min_samples_split, min_samples_leaf, n_estimators', best_params)

'Самый высокий показатель f1-меры:'

0.6094117647058824

'Лучшие значения параметров max_depth, min_samples_split, min_samples_leaf, n_estimators'

[11, 5, 1, 20]

Значение f1-меры превышает целевое. Обучим итоговую модель. Проверим метрики.

In [32]:
model_random_forest = RandomForestClassifier(random_state = 123,
                                                      max_depth = 11,
                                                      min_samples_split = 5,
                                                      min_samples_leaf = 1,
                                                      n_estimators = 20)        
model_random_forest.fit(features_train_upsampled, target_train_upsampled)
random_forest_prediction = model_random_forest.predict(features_test)
display(confusion_matrix(target_test, random_forest_prediction))
display(f1_score(target_test, random_forest_prediction))
display(roc_auc_score(target_test, random_forest_prediction))

array([[1380,  193],
       [ 141,  286]])

0.6313465783664459

0.7735468704172132

F1 на тестовой выборке выше целевого значения. Проверим модель на адекватность.

In [33]:
target_predict_constant = pd.Series([0]*len(target_test))
display('accuracy_score константой модели:', accuracy_score(target_test, target_predict_constant))
display('accuracy_score финальной модели:', accuracy_score(target_test, random_forest_prediction))

probabilities_test = model_random_forest.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
display('AUC-ROC константой модели:', roc_auc_score(target_test, target_predict_constant))
display('AUC-ROC финальной модели:', roc_auc_score(target_test, probabilities_one_test))

display('F1-score константой модели:', f1_score(target_test, target_predict_constant))
display('F1-score финальной модели:', f1_score(target_test, random_forest_prediction))

'accuracy_score константой модели:'

0.7865

'accuracy_score финальной модели:'

0.833

'AUC-ROC константой модели:'

...

0.8603185190368499

'F1-score константой модели:'

0.0

'F1-score финальной модели:'

0.6313465783664459

Метрики финальной модели выше, чему у константной, и соответствуют заданию.

Выводы:

1) Нашли подходящие гиперпараметры для финальной модели.

2) Проверили модель на тестовой выборке, с найденными гиперпараметрами. Значение F1-меры на тестовой выборке превышает 0.59, что соответствует условиям задачи. Значение AUC-ROC, как и при исследовании других моделей, также выросло вместе со значением F1-меры.

3) Сравнили результат с результатом константной модели.